# Recommender Systems

## 1. Movie Ratings dataset

In [1]:
import scipy.io

In [2]:
mat = scipy.io.loadmat('ex8_movies.mat')

In [3]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Y', 'R'])

In [4]:
Y = mat['Y'] #row: movies, col: people
Y.shape

(1682, 943)

In [5]:
R = mat['R']
R.shape

(1682, 943)

In [6]:
import numpy as np

In [7]:
index = np.argwhere(R[0]==1)
av = np.mean(Y[0, index])
print('average rating for movie 1 (toy story):',av)

average rating for movie 1 (toy story): 3.8783185840707963


In [8]:
y = np.mat(Y)

In [9]:
import cv2

In [10]:
ys = cv2.resize(y, (540,480))
ys = ys/np.max(y)
cv2.imshow('image',ys)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 2. Collaborative filtering learning algorithm

### 2.1 Collaborative filtering cost function

In [10]:
mat2 = scipy.io.loadmat('ex8_movieParams.mat')

In [11]:
mat2.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'Theta', 'num_users', 'num_movies', 'num_features'])

In [71]:
X = mat2['X']
Theta = mat2['Theta']
num_users = mat2['num_users'][0][0]
num_movies = mat2['num_movies'][0][0]
num_features = mat2['num_features'][0][0]
print('X:',X.shape)
print('Theta:',Theta.shape)
print('num_users:',num_users)
print('num_movies:',num_movies)
print('num_features:',num_features)

X: (1682, 10)
Theta: (943, 10)
num_users: 943
num_movies: 1682
num_features: 10


In [72]:
param = np.concatenate((X,Theta), axis = None)
param.shape

(26250,)

In [14]:
def cofiCostFunc(param, Y, R, num_users, num_movies, num_features, Lambda):
    X = param[:num_movies*num_features].reshape(num_movies, num_features)
    Theta = param[num_movies*num_features:].reshape(num_users, num_features)
    diff = np.dot(X,Theta.transpose()) - Y
    J = np.sum( np.square( diff*R ))*0.5 + ( np.sum(np.square(Theta)) + np.sum(np.square(X)) )*0.5*Lambda
    gradX = np.dot((diff*R),Theta) + Lambda*X
    gradTheta = np.dot((diff*R).transpose(), X) + Lambda*Theta
    return [J, np.concatenate((gradX, gradTheta), axis = None)] #make grad as one col

In [15]:
nmov = 5
nuser = 4
nfea = 3
J, grad = cofiCostFunc(np.concatenate((X[0:nmov,0:nfea], Theta[0:nuser, 0:nfea]), axis = None),\
                 Y[0:nmov, 0:nuser], R[0:nmov, 0:nuser], nuser, nmov, nfea,0)
print('cost at loaded parameters:',J)

cost at loaded parameters: 22.224603725685675


## 2.2 Collaborative filtering gradient

In [16]:
J, grad = cofiCostFunc(np.concatenate((X[0:nmov,0:nfea], Theta[0:nuser, 0:nfea]), axis = None),\
                 Y[0:nmov, 0:nuser], R[0:nmov, 0:nuser], nuser, nmov, nfea,1.5)

In [17]:
print('J:',J)
print('grad.shape:', grad.shape)

J: 31.34405624427422
grad.shape: (27,)


---
## 2.3 Gradient Checking

### 2.3.1 Create small size sets

In [18]:
num_movies = 4
num_features = 3
num_users = 5
X_t = np.random.rand(num_movies,num_features)
Theta_t = np.random.rand(num_users,num_features)
Y = np.dot(X_t, Theta_t.transpose())
print('X:',X_t)
print('Theta:',Theta_t)
print('Y:',Y)

X: [[2.67648088e-04 3.00227861e-02 3.57080014e-02]
 [7.85120343e-01 5.54527273e-01 8.93551805e-01]
 [1.48986031e-01 3.80067496e-02 3.78708516e-01]
 [1.66933446e-03 5.66761902e-01 8.13957529e-01]]
Theta: [[0.04582048 0.64844231 0.77934494]
 [0.94876083 0.24613814 0.57514885]
 [0.43551954 0.99580656 0.12246827]
 [0.54446527 0.13337267 0.26689846]
 [0.79372824 0.36371386 0.41661454]]
Y: [[0.04730916 0.0281811  0.03438655 0.01368035 0.02600862]
 [1.09193862 1.39530703 1.00356889 0.73991714 1.19712812]
 [0.32661636 0.36852079 0.14911347 0.1872635  0.28985348]
 [1.00194257 0.60923225 0.66479622 0.29374345 0.5465707 ]]


### 2.3.2 Create missing data

In [19]:
Y[Y>0.5]=0
R = np.zeros(Y.shape)
R[Y != 0]=1
print('Y:',Y)
print('R:',R)

Y: [[0.04730916 0.0281811  0.03438655 0.01368035 0.02600862]
 [0.         0.         0.         0.         0.        ]
 [0.32661636 0.36852079 0.14911347 0.1872635  0.28985348]
 [0.         0.         0.         0.29374345 0.        ]]
R: [[1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 0.]]


In [70]:
X = np.random.rand(num_movies,num_features)
Theta = np.random.rand(num_users,num_features)

### 2.3.3 Implement Gradient Checking

In [21]:
import math

In [22]:
theta = np.concatenate((X,Theta), axis = None)

In [23]:
def computeNumericalGradient(theta, num_users,num_movies, num_features, Lambda):
    numgrad = np.zeros(theta.size)
    e = math.exp(1e-4)
    perturb = np.zeros(theta.size)
    for i in range(0, theta.size):
        perturb[i] = e
        tmp = theta - perturb
        loss1, grad = cofiCostFunc(tmp, Y, R, num_users, num_movies, num_features, Lambda)
        tmp = theta + perturb
        loss2, grad = cofiCostFunc(tmp, Y, R, num_users, num_movies, num_features, Lambda)
        numgrad[i] = (loss2-loss1)/(2*e)
        perturb[i] = 0
    return numgrad

In [24]:
numgrad = computeNumericalGradient(theta, num_users, num_movies, num_features, 0)
numgrad

array([1.71918689, 2.02935985, 2.46137937, 0.        , 0.        ,
       0.        , 0.42884899, 0.95071492, 0.86170869, 0.02336735,
       0.34264356, 0.01434701, 0.16580919, 1.05484834, 0.66006073,
       0.05567418, 0.22506681, 0.24028483, 0.13259836, 0.84712514,
       0.52733951, 0.42918023, 0.99772977, 0.7368564 , 0.07004507,
       0.37225631, 0.28943672])

In [25]:
J, grad = cofiCostFunc(theta, Y, R, num_users, num_movies, num_features, 0)
grad

array([1.71918689, 2.02935985, 2.46137937, 0.        , 0.        ,
       0.        , 0.42884899, 0.95071492, 0.86170869, 0.02336735,
       0.34264356, 0.01434701, 0.16580919, 1.05484834, 0.66006073,
       0.05567418, 0.22506681, 0.24028483, 0.13259836, 0.84712514,
       0.52733951, 0.42918023, 0.99772977, 0.7368564 , 0.07004507,
       0.37225631, 0.28943672])

In [26]:
diff = np.linalg.norm(numgrad - grad)/(np.linalg.norm(numgrad+grad))
print('relative difference should be less 1e-9:  ', diff)

relative difference should be less 1e-9:   1.1806958638996733e-16


### 2.3.4 Regularized gradient checking

In [27]:
numgrad = computeNumericalGradient(theta, num_users, num_movies, num_features, 1.5)
numgrad

array([1.90815351, 2.96775019, 3.25173523, 0.11348139, 0.04713975,
       0.3179112 , 0.51187617, 2.02961668, 1.11266672, 1.13357856,
       1.81664245, 1.39594501, 1.10690813, 2.43069928, 2.03164111,
       1.50044462, 0.43437582, 1.12937016, 0.7646171 , 1.65746669,
       1.97743876, 0.50271571, 2.07600594, 0.78200548, 0.56363972,
       0.84076248, 1.2033396 ])

In [28]:
J, grad = cofiCostFunc(theta, Y, R, num_users, num_movies, num_features, 1.5)
grad

array([1.90815351, 2.96775019, 3.25173523, 0.11348139, 0.04713975,
       0.3179112 , 0.51187617, 2.02961668, 1.11266672, 1.13357856,
       1.81664245, 1.39594501, 1.10690813, 2.43069928, 2.03164111,
       1.50044462, 0.43437582, 1.12937016, 0.7646171 , 1.65746669,
       1.97743876, 0.50271571, 2.07600594, 0.78200548, 0.56363972,
       0.84076248, 1.2033396 ])

In [29]:
diff = np.linalg.norm(numgrad - grad)/(np.linalg.norm(numgrad+grad))
print('relative difference should be less 1e-9:  ', diff)

relative difference should be less 1e-9:   1.9924960018067223e-16


---
## 2.4 Learning movie recommendations

### 2.4.1 Add my ratings

In [73]:
Y = mat['Y'] #row: movies, col: people
Y.shape

(1682, 943)

In [74]:
R = mat['R'] #row: movies, col: people
R.shape

(1682, 943)

In [32]:
f = open("movie_ids.txt", mode ="r", encoding="ISO-8859-1")

In [33]:
movielist = []
for s in f.readlines():
    movielist.append(s.strip('\n'))

In [34]:
movielist

['1 Toy Story (1995)',
 '2 GoldenEye (1995)',
 '3 Four Rooms (1995)',
 '4 Get Shorty (1995)',
 '5 Copycat (1995)',
 '6 Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 '7 Twelve Monkeys (1995)',
 '8 Babe (1995)',
 '9 Dead Man Walking (1995)',
 '10 Richard III (1995)',
 '11 Seven (Se7en) (1995)',
 '12 Usual Suspects, The (1995)',
 '13 Mighty Aphrodite (1995)',
 '14 Postino, Il (1994)',
 "15 Mr. Holland's Opus (1995)",
 '16 French Twist (Gazon maudit) (1995)',
 '17 From Dusk Till Dawn (1996)',
 '18 White Balloon, The (1995)',
 "19 Antonia's Line (1995)",
 '20 Angels and Insects (1995)',
 '21 Muppet Treasure Island (1996)',
 '22 Braveheart (1995)',
 '23 Taxi Driver (1976)',
 '24 Rumble in the Bronx (1995)',
 '25 Birdcage, The (1996)',
 '26 Brothers McMullen, The (1995)',
 '27 Bad Boys (1995)',
 '28 Apollo 13 (1995)',
 '29 Batman Forever (1995)',
 '30 Belle de jour (1967)',
 '31 Crimson Tide (1995)',
 '32 Crumb (1994)',
 '33 Desperado (1995)',
 '34 Doom Generation, The (1995)',
 '35

In [35]:
n = len(movielist)
my_ratings = np.zeros((n,1))
my_ratings.shape

(1682, 1)

In [36]:
my_ratings[0]=4
my_ratings[97]=2
my_ratings[6]=3
my_ratings[11]=5
my_ratings[53]=4
my_ratings[63]=5
my_ratings[65]=3
my_ratings[68]=5
my_ratings[182]=4
my_ratings[225]=5
my_ratings[354]=5

In [37]:
for i in range(0,n):
    if(my_ratings[i] > 0):
        print('Rated:',my_ratings[i],' for ', movielist[i])

Rated: [4.]  for  1 Toy Story (1995)
Rated: [3.]  for  7 Twelve Monkeys (1995)
Rated: [5.]  for  12 Usual Suspects, The (1995)
Rated: [4.]  for  54 Outbreak (1995)
Rated: [5.]  for  64 Shawshank Redemption, The (1994)
Rated: [3.]  for  66 While You Were Sleeping (1995)
Rated: [5.]  for  69 Forrest Gump (1994)
Rated: [2.]  for  98 Silence of the Lambs, The (1991)
Rated: [4.]  for  183 Alien (1979)
Rated: [5.]  for  226 Die Hard 2 (1990)
Rated: [5.]  for  355 Sphere (1998)


In [38]:
myY = np.concatenate((my_ratings, Y), axis = 1)
myY.shape

(1682, 944)

In [39]:
myR = np.concatenate((my_ratings!=0,R), axis =1)
myR.shape

(1682, 944)

### 2.4.2 Mean Normalization

In [169]:
def normalizeRatings(myY, myR):
    [m,n] = myY.shape
    Ymean = np.zeros(m)
    Ynorm = np.zeros(myY.shape)
    for i in range(m):
        idx = np.argwhere(myR[i,:]==1)
        Ymean[i] = np.mean(myY[i,idx])
        Ynorm[i,idx] = myY[i,idx]-Ymean[i]
    return Ynorm, Ymean

In [170]:
Ynorm, Ymean = normalizeRatings(myY,myR)
print('Ymean:',Ymean)
print('Ynorm:',Ynorm)

Ymean: [3.8785872  3.20610687 3.03333333 ... 2.         3.         3.        ]
Ynorm: [[ 0.1214128   1.1214128   0.1214128  ...  1.1214128   0.
   0.        ]
 [ 0.         -0.20610687  0.         ...  0.          0.
   1.79389313]
 [ 0.          0.96666667  0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [183]:
min(Ymean)

1.0

### 2.4.3 Recommendations

In [42]:
from scipy.optimize import minimize

In [43]:
my_movies, my_users = myY.shape
my_features = 10

In [113]:
myX = np.random.randn(my_movies, my_features)
myTheta = np.random.randn(my_users, my_features)
print('myX:',myX)
print('myTheta:',myTheta)

myX: [[-0.39442739 -1.37746731  0.73491529 ... -1.11012938  1.05867352
   1.48312038]
 [ 0.28659989 -1.69150673  1.35201561 ...  1.20019094  1.10803744
  -0.7325916 ]
 [-1.44734469 -1.20460352  0.84733893 ... -0.0069134   0.34932414
   1.73716463]
 ...
 [-0.67009074 -0.70308014 -0.65649663 ...  0.89170364 -0.26974247
  -0.40256915]
 [ 0.13342336 -0.07403143  1.21119138 ... -0.17149504  0.12176627
  -0.80373288]
 [-1.11010611 -1.32351084 -1.36521547 ... -0.68507732 -0.9897282
  -2.93649583]]
myTheta: [[-0.17645992  0.79972968 -0.02551817 ... -0.33128298 -1.39994226
   0.05204392]
 [-0.91166244  0.37287782 -1.33387789 ... -0.28674518 -0.01975678
  -0.69039815]
 [ 0.00966269 -0.24653575  1.07594285 ... -0.15346142  0.50586857
   0.73230919]
 ...
 [ 2.12554406  1.10087126  0.54936235 ...  0.97955769 -1.02104806
   0.37506309]
 [ 0.83337862  1.30201726  0.06017233 ...  0.07717914  0.21911659
  -0.76982245]
 [-0.29830953  0.63955076 -0.17240468 ...  1.11343261 -1.1183957
   1.46877526]]


In [114]:
initial_parameters = np.concatenate((myX, myTheta), axis = None)

In [330]:
#don't know why using Ymean as y will get total different answer
result = minimize(lambda x: cofiCostFunc(x, myY, myR, my_users, my_movies, num_features, 10),\
                 initial_parameters, method='TNC',jac=True, options={'maxiter':500})

In [331]:
print(result)

     fun: 71897.41749177361
     jac: array([ 4.07968585e-03, -2.00888698e-02,  3.59187673e-03, ...,
        5.22604923e-03, -2.09857960e-03,  9.36582949e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 500
     nit: 30
  status: 3
 success: False
       x: array([-0.25997809, -0.63589379,  0.19953173, ..., -0.94355643,
        0.68016955,  0.17946095])


In [332]:
result.x.shape

(26260,)

In [333]:
ans_x = result.x[0:my_movies*my_features].reshape(my_movies, my_features)
ans_x.shape

(1682, 10)

In [334]:
ans_theta = result.x[my_movies*my_features:].reshape(my_users,my_features)
ans_theta.shape

(944, 10)

In [335]:
ans_x

array([[-0.25997809, -0.63589379,  0.19953173, ..., -0.99136213,
         0.29809314,  0.22025841],
       [-0.08172972, -0.21245998, -0.078863  , ..., -0.804466  ,
         0.49729932,  0.66139007],
       [-0.38070341,  0.60408896, -0.76803397, ..., -0.11059701,
         0.46876071,  0.95928832],
       ...,
       [-0.06081948,  0.11058272, -0.14518274, ..., -0.02241841,
         0.04529435,  0.01461607],
       [ 0.06655965,  0.05147523,  0.01220169, ..., -0.2012451 ,
         0.13295311,  0.03301696],
       [-0.18639441,  0.17820418, -0.03130577, ..., -0.16762943,
         0.04802881,  0.08488476]])

In [336]:
p = np.dot(ans_x, ans_theta.T)
p.shape

(1682, 944)

In [337]:
my_prediction = p[:,0]

In [338]:
my_index =(-my_prediction).argsort() #sort for highest rating

In [339]:
my_index

array([ 312,   49,  173, ..., 1564, 1568, 1547], dtype=int64)

In [340]:
for i in range(0,10):
    index = my_index[i]
    print('Predictin rating:',my_prediction[index],' movie:',movielist[index])

Predictin rating: 4.296488843908813  movie: 313 Titanic (1997)
Predictin rating: 4.120684191785352  movie: 50 Star Wars (1977)
Predictin rating: 3.9775578305716333  movie: 174 Raiders of the Lost Ark (1981)
Predictin rating: 3.912564625512516  movie: 272 Good Will Hunting (1997)
Predictin rating: 3.884974532986415  movie: 64 Shawshank Redemption, The (1994)
Predictin rating: 3.8727098269146056  movie: 181 Return of the Jedi (1983)
Predictin rating: 3.8701015483785652  movie: 22 Braveheart (1995)
Predictin rating: 3.8622947119880644  movie: 172 Empire Strikes Back, The (1980)
Predictin rating: 3.7586565649398684  movie: 96 Terminator 2: Judgment Day (1991)
Predictin rating: 3.757235046173744  movie: 316 As Good As It Gets (1997)
